# Calculate 24

Calculate 24 from given numbers using +, -, *, /.

In [22]:
from typing import Callable, Optional, Collection, Iterable, List, Set
import itertools
from fractions import Fraction
from operator import add, sub, mul, truediv

class Expr:
    def __init__(self, lhs: Optional['Expr'] = None, op: Optional[Callable] = None, rhs: Optional['Expr'] = None, *, value: Optional[Fraction] = None) -> None:
        if op is not None:
            assert(lhs is not None and rhs is not None)
            self.value = op(lhs.value, rhs.value)
        else:
            assert(lhs is None and rhs is None and value is not None)
            self.value = value
        self.op = op
        self.lhs = lhs
        self.rhs = rhs
    
    def __add__(self, other: 'Expr') -> 'Expr':
        return Expr(self, add, other)
    def __sub__(self, other: 'Expr') -> 'Expr':
        return Expr(self, sub, other)
    def __mul__(self, other: 'Expr') -> 'Expr':
        return Expr(self, mul, other)
    def __truediv__(self, other: 'Expr') -> 'Expr':
        return Expr(self, truediv, other)
    
    def __lt__(self, other: 'Expr') -> bool:
        if self.value != other.value:
            return self.value < other.value
        else:
            order = [add, sub, mul, truediv, None]
            if order.index(self.op) != order.index(other.op):
                return order.index(self.op) < order.index(other.op)
            elif self.lhs is None and other.lhs is None:
                return self.value < other.value
            elif self.lhs is None:
                return False
            elif other.lhs is None:
                return True
            elif self.lhs != other.lhs:
                return self.lhs < other.lhs
            else:
                assert(self.rhs is not None and other.rhs is not None)
                return self.rhs < other.rhs
    def __gt__(self, other: 'Expr') -> bool:
        return other < self
    def __eq__(self, other: 'Expr') -> bool:
        return self.value == other.value and self.op == other.op and self.lhs == other.lhs and self.rhs == other.rhs
    def __ne__(self, other: 'Expr') -> bool:
        return not (self == other)
    def __le__(self, other: 'Expr') -> bool:
        return not (other < self)
    def __ge__(self, other: 'Expr') -> bool:
        return not (self < other)

    def __repr__(self) -> str:
        if self.op is None:
            return str(self.value)
        else:
            assert(self.lhs is not None and self.rhs is not None)
            if self.op == add:
                lhs_str = f'{self.lhs}'
                rhs_str = f'{self.rhs}'
                if self.lhs > self.rhs:
                    lhs_str, rhs_str = rhs_str, lhs_str
                return f'{self.lhs}+{self.rhs}'
            elif self.op == sub:
                rhs_str = f'({self.rhs})' if self.rhs.op in [add, sub] else f'{self.rhs}'
                return f'{self.lhs}-{rhs_str}'
            elif self.op == mul:
                lhs_str = f'({self.lhs})' if self.lhs.op in [add, sub] else f'{self.lhs}'
                rhs_str = f'({self.rhs})' if self.rhs.op in [add, sub] else f'{self.rhs}'
                if self.lhs > self.rhs:
                    lhs_str, rhs_str = rhs_str, lhs_str
                return f'{lhs_str}*{rhs_str}'
            elif self.op == truediv:
                lhs_str = f'({self.lhs})' if self.lhs.op in [add, sub] else f'{self.lhs}'
                rhs_str = f'({self.rhs})' if self.rhs.op in [add, sub, truediv] else f'{self.rhs}'
                return f'{lhs_str}/{rhs_str}'
            else:
                raise ValueError


def build_expr(candidates: List[Expr]) -> List[Expr]:
    '''
    choose from candidates and build them to one expression
    return all the possible results
    '''
    if len(candidates) == 1:
        return list(candidates)

    result: List[Expr] = []
    for i, j in itertools.combinations(range(len(candidates)), 2):
        new_candidates = list(candidates)
        ej = new_candidates.pop(j) # i < j
        ei = new_candidates.pop(i)
        new_candidates.insert(0, ei + ej)
        result += build_expr(new_candidates)
        new_candidates[0] = ei - ej
        result += build_expr(new_candidates)
        new_candidates[0] = ej - ei
        result += build_expr(new_candidates)
        new_candidates[0] = ei * ej
        result += build_expr(new_candidates)
        try:
            new_candidates[0] = ei / ej
            result += build_expr(new_candidates)
        except ZeroDivisionError:
            pass
        try:
            new_candidates[0] = ej / ei
            result += build_expr(new_candidates)
        except ZeroDivisionError:
            pass
    return result


def calc_24(arr: Iterable[int], target=24):
    return list(set([str(expr) for expr in build_expr([Expr(value=Fraction(x)) for x in arr]) if expr.value == target]))


In [23]:
for eq in calc_24([3, 3, 7, 7]):
    print(eq)

(3/7+3)*7


In [24]:
for eq in calc_24([8, 8, 10, 3]):
    print(eq)

(8*10-8)/3


In [25]:
for eq in calc_24([1, 1, 3, 5]):
    print(eq)

(1+3)*(1+5)


In [35]:
for eq in calc_24([1, 4, 5, 6]):
    print(eq)

6/(5/4-1)
4/(1-5/6)
